# Postgres database for the full dataset

Takes the full dataset created in pandas, stores it to postgres.
Also reads the feature counts csv file and prints a check that counts for each label match.

In [14]:
import pandas as pd

In [15]:
%load_ext sql

In [16]:
%load_ext watermark
%watermark -iv

pandas 0.24.0



In [17]:
# only need to do this once! 
# !psql -c "create database nb15;"

In [18]:
%sql postgres://localhost/nb15

'Connected: @nb15'

In [7]:
full_data = pd.read_pickle('../data/full_pandas_final')

In [8]:
full_data.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,﻿59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,normal,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,normal,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,normal,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,normal,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,normal,0


In [9]:
full_data.shape

(2540044, 49)

In [10]:
%sql persist full_data

 * postgres://localhost/nb15


'Persisted full_data'

In [19]:
%sql select count(*) from full_data;

 * postgres://localhost/nb15
1 rows affected.


count
2540044


In [20]:
%sql select * from full_data limit 5

 * postgres://localhost/nb15
5 rows affected.


index,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,sload,dload,spkts,dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,sjit,djit,stime,ltime,sintpkt,dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,﻿59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.9375,621800.9375,2,2,0,0,0,0,66,82,0,0,0.0,0.0,1421927414,1421927414,0.017,0.013,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,normal,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,normal,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.5313,636282.375,2,2,0,0,0,0,73,89,0,0,0.0,0.0,1421927414,1421927414,0.017,0.013,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,normal,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.5625,542597.1875,2,2,0,0,0,0,66,82,0,0,0.0,0.0,1421927414,1421927414,0.043,0.014,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,normal,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25,609067.5625,2,2,0,0,0,0,73,89,0,0,0.0,0.0,1421927414,1421927414,0.005,0.003,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,normal,0


In [21]:
%sql select attack_cat, count(*) from full_data group by attack_cat;

 * postgres://localhost/nb15
10 rows affected.


attack_cat,count
analysis,2677
backdoors,2329
dos,16353
exploits,44525
fuzzers,24246
generic,215481
normal,2218761
reconnaissance,13987
shellcode,1511
worms,174


In [22]:
import pandas as pd
event_check = pd.read_csv('../data/UNSW-NB15_LIST_EVENTS.csv')

In [23]:
event_check.groupby('Attack category')['Number of events'].sum()

Attack category
 Fuzzers               1405
 Fuzzers              22841
 Reconnaissance       12228
 Shellcode             1288
Analysis               2677
Backdoors              2329
DoS                   16353
Exploits              44525
Generic              215481
Reconnaissance         1759
Shellcode               223
Worms                   174
normal              2218761
Name: Number of events, dtype: int64

In [24]:
event_check['Attack category'] = event_check['Attack category'].str.replace(' ','')

In [25]:
event_check.groupby('Attack category')['Number of events'].sum()

Attack category
Analysis             2677
Backdoors            2329
DoS                 16353
Exploits            44525
Fuzzers             24246
Generic            215481
Reconnaissance      13987
Shellcode            1511
Worms                 174
normal            2218761
Name: Number of events, dtype: int64